In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.getcwd()

'D:\\workspace\\DACON_2019\\SUHYUN'

In [3]:
os.chdir('D:\\workspace\\DACON_2019')

In [4]:
test = pd.read_csv('.\\SOOMIN\\data\\Test_시간별평균대치.csv')
test

,Time,X26,X303,X241,X435,X402,X352,X305,X350,X326,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
0,2017-07-01 00:00:00,1.187315,0.385586,0.298691,0.276657,0.190045,0.160754,0.343880,0.249761,0.070329,...,0.288141,0.326095,0.356141,0.407326,0.159644,0.227077,0.168175,0.275,0.021,0.338138
1,2017-07-01 01:00:00,1.186123,0.478955,0.307983,0.333324,0.193712,0.167303,0.426744,0.293231,0.072641,...,0.259911,0.297695,0.296728,0.316149,0.159288,0.259118,0.184568,0.222,0.021,0.335306
2,2017-07-01 02:00:00,1.168301,0.467760,0.323560,0.331767,0.200656,0.165983,0.405878,0.301436,0.076477,...,0.237175,0.261525,0.288334,0.267332,0.157027,0.267059,0.177599,0.237,0.021,0.344922
3,2017-07-01 03:00:00,1.163425,0.443529,0.304879,0.322123,0.189067,0.155355,0.401216,0.288693,0.082617,...,0.203332,0.207759,0.253232,0.212221,0.154466,0.251081,0.156110,0.229,0.020,0.334552
4,2017-07-01 04:00:00,1.398466,0.390286,0.322174,0.320922,0.195500,0.154073,0.344145,0.295861,0.074067,...,0.189208,0.174952,0.266133,0.244444,0.152014,0.253930,0.143873,0.202,0.021,0.346241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2018-06-30 19:00:00,1.255000,0.175000,0.430000,0.429000,0.169000,0.739000,0.573000,0.341000,0.127000,...,0.312000,0.381000,0.479000,0.858000,11.546000,0.428000,0.134000,3.519,0.027,0.286000
8756,2018-06-30 20:00:00,1.596000,1.692000,0.332000,0.366000,0.159000,0.255000,0.501000,0.281000,0.126000,...,0.327000,0.402000,0.725000,0.816000,11.591000,0.413000,0.206000,3.331,0.027,0.303000
8757,2018-06-30 21:00:00,1.379000,0.883000,0.435000,0.497000,0.166000,0.233000,0.634000,0.401000,0.439000,...,0.253000,0.401000,0.426000,0.659000,10.766000,0.466000,0.324000,3.416,0.026,0.253000
8758,2018-06-30 22:00:00,1.000000,0.539000,0.268000,0.459000,0.463000,0.454000,0.676000,0.543000,0.737000,...,0.332000,0.401000,0.353000,0.462000,0.312000,0.473000,0.402000,4.263,0.027,0.253000


In [5]:
test['Time'] = pd.to_datetime(test['Time']) 
test = test.set_index('Time')

In [6]:
Series_data = test.iloc[:, 0]

In [8]:
making_shift_count = 25

dataset = Series_data.to_frame()

from sklearn.preprocessing import MinMaxScaler
dataset_sc=MinMaxScaler().fit_transform(dataset)
dataset_sc_df = pd.DataFrame(dataset_sc, columns=dataset.columns, index = dataset.index)

for s in range(1, making_shift_count):
    dataset_sc_df['shift_{}'.format(s)] = dataset_sc_df[dataset.columns].shift(s)

X_dataset = dataset_sc_df.dropna().drop(dataset.columns, axis=1)
y_dataset = dataset_sc_df.dropna()[dataset.columns]

X_dataset = X_dataset.values
y_dataset = y_dataset.values

X_dataset_t = X_dataset.reshape(X_dataset.shape[0], making_shift_count-1, 1)

In [25]:
del model

In [26]:
from keras.layers import LSTM 
from keras.models import Sequential 
from keras.layers import Dense 
import keras.backend as K 
from keras.callbacks import EarlyStopping

In [27]:
K.clear_session()
    
model = Sequential() # Sequeatial Model 
model.add(LSTM(20, input_shape=(X_dataset_t.shape[1], 1))) # (timestep, feature) 
model.add(Dense(1)) # output = 1 
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy']) 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20)                1760      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 21        
Total params: 1,781
Trainable params: 1,781
Non-trainable params: 0
_________________________________________________________________


In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=1, verbose=1)

history = model.fit(X_dataset_t, y_dataset, epochs=100, batch_size=30, verbose=1, callbacks=[early_stop])

Epoch 1/100
8736/8736 [==============================] - 7s 807us/step - loss: 0.0046 - acc: 2.2894e-04
Epoch 2/100
 240/8736 [..............................] - ETA: 5s - loss: 0.0032 - acc: 0.0000e+00

C:\Anaconda3\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


8736/8736 [==============================] - 6s 639us/step - loss: 0.0037 - acc: 2.2894e-04
Epoch 3/100
8736/8736 [==============================] - 5s 543us/step - loss: 0.0035 - acc: 2.2894e-04
Epoch 4/100
8736/8736 [==============================] - 5s 562us/step - loss: 0.0034 - acc: 2.2894e-04 0s - loss: 0.0035 
Epoch 5/100
8736/8736 [==============================] - 5s 552us/step - loss: 0.0030 - acc: 2.2894e-04
Epoch 6/100
8736/8736 [==============================] - 5s 534us/step - loss: 0.0025 - acc: 2.2894e-04
Epoch 7/100
8736/8736 [==============================] - 5s 550us/step - loss: 0.0023 - acc: 3.4341e-04 1s - loss: 0.0024 - acc: 1.7825e- - ETA: 1s - loss: 0.0024 - acc: 1.7271e - ETA: 1s - loss: 0.0024 - acc: 1.6420e-0 - ETA: 1s - loss: 0.0024 - acc: - ETA: 0s - loss: 0.0024 - acc
Epoch 8/100
8736/8736 [==============================] - 5s 553us/step - loss: 0.0021 - acc: 3.4341e-04
Epoch 9/100
8736/8736 [==============================] - 5s 552us/step - loss: 0.0020 

In [ ]:
y_vloss=history.history['val_loss'] # 오차 표시를 위해 저장
y_acc=history.history['acc'] # 정확도 표시를 위해 저장

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
x_len = np.arange(len(y_acc))
plt.plot(x_len, y_vloss, "o", c="red", markersize=3) # 빨간색은 오차
plt.plot(x_len, y_acc, "o", c="blue", markersize=3) # 파란색은 정확도
plt.show()